In [ ]:
Update

In [1]:
def twitter(names, counts, dates):
    
    consumer_key = 'aaaaaaaa'
    consumer_secret = ''
    access_token = ''
    access_token_secret = ''
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    
    all_tweets = []
    error_names = []
    outtweets = [[]]
    T = True
    
    for i in range(len(names)):
        user_tweets = []
        
        try:
            try:
                cn = int(counts[i])
            except:
                cn = 20
                
            cn_new = cn
            if cn > 200:
                cn_new = 200
                
            tweets = api.user_timeline(screen_name = names[i], count=cn_new, include_rts = False, tweet_mode = 'extended')
            user_tweets.extend(tweets)
            oldest_id = tweets[-1].id
            cn_new = cn - 200
            
            if cn_new > 0:
                while cn_new > 0:
                    cn_new2 = cn_new
                    if cn_new > 200:
                        cn_new2 = 200
                    tweets = api.user_timeline(screen_name = names[i],
                                               count=cn_new2,
                                               include_rts = False,
                                               max_id = oldest_id - 1,
                                               tweet_mode = 'extended')
                    
                    if len(tweets) == 0:
                        break
                        
                    oldest_id = tweets[-1].id
                    user_tweets.extend(tweets)
                    cn_new = cn_new - 200
                    
            all_tweets.extend(user_tweets)
            outtweets = [[tweet.user.name, 
                          tweet.created_at.strftime("%m/%d/%Y"),
                          tweet.created_at.strftime("%H:%M:%S"),
                          tweet.favorite_count,
                          tweet.full_text.encode("utf-8").decode("utf-8")] 
                         for idx,tweet in enumerate(all_tweets)]
        except:
            T = False
            error_names.append(names[i])
            
    if outtweets == [[]]:
        df = pd.DataFrame(columns=["User", "Date", "Time", "Tweet"])
        
    elif len(outtweets) == 0:
        df = pd.DataFrame(columns=["User", "Date", "Time", "Tweet"])
        
    else:
        df = pd.DataFrame(outtweets,columns=["User", "Date", "Time", "Likes Number", "Tweet"])
        names = [i for i in df.User.unique()]
        df_new = pd.DataFrame()
        
        for u,d in zip(names,dates):
            df_s = df[df.User == u]
            df_s = df_s[pd.to_datetime(df_s['Date'])>=pd.to_datetime(d)]
            df_new = pd.concat([df_s, df_new])
            
        df = df_new
        
    return df, T, error_names

In [2]:
from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog
from tkinter import messagebox
import pandas as pd
from tkcalendar import Calendar,DateEntry
from datetime import datetime
import tweepy

bg = 'rosybrown1'
entry_bg = 'white smoke'

root = Tk()
root.geometry("500x400")
root.configure(background=bg)
root.title('twitterholic')
root.iconbitmap('twitter.ico')

empty_label = Label(root, text = "", padx=20, pady=5, background=bg)
empty_label.pack()

twitter_image = Image.open("twitter.png")
twitter_image = twitter_image.resize((48, 38), Image.ANTIALIAS)
twitter_image = ImageTk.PhotoImage(twitter_image)
image = Label(image = twitter_image, padx=10, pady=10, bg=bg)
image.pack()

enter_label = Label(root, text = "Please enter the user screen names you want to extract tweets from!",
                    padx=40, pady=20, background=bg)
enter_label.pack()

e = Entry(root, width=30, fg='black', bg=entry_bg)
e.insert(0, "Enter screen name here please!")
e.pack()

text_count_label = "Please insert below the number of tweets you want to extract!"
count_label = Label(root, text = text_count_label, padx=40, pady=20, background=bg)
count_label.pack()

cn = Entry(root, width=45, fg='black', bg=entry_bg)
cn.insert(0, "Enter the number of tweets here!")
cn.pack()

date_label = Label(root, text = "Please choose the date to extract from!", padx=40, pady=20, background=bg)
date_label.pack()

year = int(datetime.today().strftime('%Y'))
month = int(datetime.today().strftime('%m'))
day = int(datetime.today().strftime('%d'))
cal = DateEntry(root,fg='white', background='black', width=30, year=year, month=month, day=day)
cal.pack()

L = []
v = 0
counts = []
dates = []
df = pd.DataFrame()

def e_callback(event):
    e.delete(0, "end")
    
def cn_callback(event): 
    cn.delete(0, "end")
    
e.bind("<Button-1>", e_callback)
cn.bind("<Button-1>", cn_callback)

def save_file(df):
    result = filedialog.asksaveasfilename()
    if result != '':
        filename = result + '.xlsx'
        df.to_excel(filename, index=False, encoding='utf-8')
        
def ask_verif():
    global enter, L, text_label, e, v, counts, cn, dates, df
    
    dates.append(cal.get())
    L.append(e.get())
    counts.append(cn.get())
    
    e.bind("<Button-1>", e_callback)
    question_text = "Do you want to add another user name?\nPlease wait for a moment if you choose no!"
    response = messagebox.askquestion("", question_text)
    
    if response == 'no':
        df_new, T, error_names = twitter(L, counts, dates)
        df = pd.concat([df,df_new])
        if T == False:
            n = len(error_names)
            if n == 1:
                if v != 0:
                    text_label.destroy()
                    
                root.geometry("500x520")
                text = str(n) + ' name is invalid: \n' + str(error_names[0]) + '\n Please retry the wrong name!'
                text_label = Label(root, text = text, padx=40, pady=20, background=bg)
                text_label.pack()
                L = []
                counts = []
                dates = []
                v = 1
                
            else:
                if v != 0:
                    text_label.destroy()

                ch = ''
                for c in error_names:
                    ch += str(c) + '\n'
                root.geometry("500x600")
                text = str(n) + ' names are invalid: \n' + ch + '\n Please retry all the wrong names!'
                text_label = Label(root, text = text, padx=40, pady=20, background=bg)
                text_label.pack()
                L = []
                counts = []
                dates = []
                v = 1
                
        else:
            e.destroy()
            cn.destroy()
            enter.destroy()
            count_label.destroy()
            date_label.destroy()
            cal.destroy()
            if v!=0:
                text_label.destroy()
            enter_label.destroy()
            empty_label2.destroy()
            root.geometry("500x200")
            file_ready_text = "Your file is ready, please name and save it in your machine"
            text_label = Label(root, text = file_ready_text, padx=40, pady=20, background=bg)
            text_label.pack()
            save_button = Button(root, text = "Save file",
                                 command=lambda: save_file(df), bg='steelblue', fg='white', height = 1, width = 8)
            save_button.pack()

empty_label2 = Label(root, text = "", padx=20, pady=5, background=bg)
empty_label2.pack()

enter = Button(root, text='Enter', command=ask_verif, bg='steelblue', fg='white', height = 2, width = 8)
enter.pack()

close_button = Button(root, text = "Exit", command=root.destroy, bg='red4', fg='white', height = 1, width = 5)
close_button.pack(side=BOTTOM, anchor=E)

root.mainloop()